<a href="https://colab.research.google.com/github/limweiliang/weed-classifier/blob/main/CS3244_ProjectCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tfds-nightly tensorflow-datasets tensorflow matplotlib

     |████████████████████████████████| 4.0 MB 5.2 MB/s 


In [2]:
!nvidia-smi

Sat Oct 23 01:36:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sklearn
import math

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
train_ds, validation_ds, test_ds = tfds.load('deep_weeds', as_supervised= True, shuffle_files= True, split=["train[:70%]", "train[:20%]", "train[:10%]"], batch_size = 128)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/17509 [00:00<?, ? examples/s]

Shuffling deep_weeds-train.tfrecord...:   0%|          | 0/17509 [00:00<?, ? examples/s]

Dataset deep_weeds downloaded and prepared to /root/tensorflow_datasets/deep_weeds/3.0.0. Subsequent calls will reuse this data.
Number of training samples: 96
Number of validation samples: 28
Number of test samples: 14


In [5]:
from tensorflow.keras import datasets, layers, models, regularizers
import datetime

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(96, (11, 11), activation='relu', input_shape=(256, 256, 3), strides = 4))
model.add(layers.MaxPooling2D((3, 3), strides = 2))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3), strides = 2))

model.add(layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3), strides = 2))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(9))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 62, 62, 96)        34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 96)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 96)        384       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 192)       442560    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 192)      

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])




model.fit(train_ds, epochs=50, 
                    validation_data=validation_ds, batch_size = 128)

Epoch 1/50
96/96 [==============================] - 38s 334ms/step - loss: 1.9495 - accuracy: 0.5144 - val_loss: 4.1399 - val_accuracy: 0.0620
Epoch 2/50
96/96 [==============================] - 27s 281ms/step - loss: 1.3064 - accuracy: 0.5450 - val_loss: 7.8369 - val_accuracy: 0.3818
Epoch 3/50
96/96 [==============================] - 27s 281ms/step - loss: 1.2129 - accuracy: 0.5655 - val_loss: 1.4859 - val_accuracy: 0.5317
Epoch 4/50
96/96 [==============================] - 27s 281ms/step - loss: 1.1594 - accuracy: 0.5831 - val_loss: 1.2904 - val_accuracy: 0.5740
Epoch 5/50
96/96 [==============================] - 27s 280ms/step - loss: 1.0850 - accuracy: 0.6119 - val_loss: 1.2760 - val_accuracy: 0.5800
Epoch 6/50
96/96 [==============================] - 27s 281ms/step - loss: 1.0299 - accuracy: 0.6257 - val_loss: 2.3705 - val_accuracy: 0.4495
Epoch 7/50
96/96 [==============================] - 27s 280ms/step - loss: 0.9707 - accuracy: 0.6496 - val_loss: 1.4190 - val_accuracy: 0.5191

In [ ]:
y_pred = model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1) # Convert one-hot to index

In [ ]:
test = tfds.as_numpy(test_ds)
y_pred = []
y_label = []

for i in test:
  flat_img = i[0]
  flat_label = i[1]
  pred = model.predict(flat_img)
  pred = np.argmax(pred, axis=1)
  y_pred.extend(pred)
  y_label.extend(flat_label)

y_pred = np.array(y_pred)
y_label = np.array(y_label)

In [ ]:
len(y_pred)

1751

In [ ]:
from sklearn.metrics import classification_report

print(y_pred)
print(classification_report(y_label, y_pred))

[1 7 7 ... 8 0 8]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        98
           1       0.95      0.97      0.96       108
           2       0.98      0.83      0.90       101
           3       0.97      0.91      0.94       114
           4       0.83      0.97      0.89        91
           5       0.97      0.97      0.97       109
           6       0.96      0.98      0.97       105
           7       0.99      0.95      0.97        98
           8       0.97      0.98      0.98       927

    accuracy                           0.96      1751
   macro avg       0.96      0.95      0.95      1751
weighted avg       0.96      0.96      0.96      1751



IndentationError: ignored